# MLOps End-to-End Workflow

This notebook provides a step-by-step guide to run the entire machine learning pipeline, from training the model to deploying it as a containerized API.

## Step 1: Install Dependencies

First, we need to install all the required Python libraries listed in the `requirements.txt` file. This ensures our environment is set up correctly.

In [1]:
!pip install -r requirements.txt

## Step 2: Train the Model

Next, we run the `train.py` script. This script will:
1. Load and preprocess the Iris dataset.
2. Train multiple classification models (e.g., Logistic Regression, Random Forest).
3. Use MLflow to track experiments, parameters, and metrics.
4. Select the best-performing model and save it locally to the `saved_model/` directory.

In [2]:
!python src/train.py

   sepal_length  sepal_width  petal_length  petal_width  species
0           5.1          3.5           1.4          0.2        0
1           4.9          3.0           1.4          0.2        0
2           4.7          3.2           1.3          0.2        0
3           4.6          3.1           1.5          0.2        0
4           5.0          3.6           1.4          0.2        0
Confusion Matrix for Logistic Regression:
[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]

Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30


Precision for Logistic Regression: 1.0000
Recall for Logistic Regression: 1.0000
F1 Score for Logisti

2025/08/11 19:51:20 INFO mlflow.tracking.fluent: Experiment with name 'Iris-Classifier-Training' does not exist. Creating a new experiment.
2025/08/11 19:51:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/11 19:51:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/11 19:51:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/11 19:51:32 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/08/11 19:51:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/11 19:51:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/11 19:51:45 WARNING ml

## Step 3: Build the Docker Image

Now that we have a trained model saved in dir, we can build our Docker image. The `docker build` command reads the `dockerfile` to create a self-contained image that includes our FastAPI application `app.py`, all dependencies, and the saved model.

In [3]:
!docker build -t iris-classifier:latest .

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from dockerfile
#1 transferring dockerfile: 656B 0.0s done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/library/python:3.11-slim
#2 ...

#3 [auth] library/python:pull token for registry-1.docker.io
#3 DONE 0.0s

#2 [internal] load metadata for docker.io/library/python:3.11-slim
#2 DONE 2.5s

#4 [internal] load .dockerignore
#4 transferring context: 2B 0.0s done
#4 DONE 0.0s

#5 [1/6] FROM docker.io/library/python:3.11-slim@sha256:0ce77749ac83174a31d5e107ce0cfa6b28a2fd6b0615e029d9d84b39c48976ee
#5 DONE 0.0s

#6 [internal] load build context
#6 transferring context: 2.70kB 0.0s done
#6 DONE 0.0s

#7 [4/6] RUN pip install --no-cache-dir -r requirements.txt
#7 CACHED

#8 [5/6] COPY ./src /app/src
#8 CACHED

#9 [2/6] WORKDIR /app
#9 CACHED

#10 [3/6] COPY requirements.txt .
#10 CACHED

#11 [6/6] COPY ./saved_model /app/saved_model
#11 CACHED

#12 exporting to image
#12 exporting lay

## Step 4: Run the Docker Container

With the image successfully built, we can run it as a container. The `docker run` command starts our FastAPI application.

- `-d`: Runs the container in detached mode .
- `-p 8000:8000`: Maps port 8000 on our local machine to port 8000 inside the container.
- `--name iris_api_service`: Assigns a convenient name to our container.
- RETRAIN_API_KEY="aks" : For Secure API
- GH_PAT : Github Personal Access Token

In [ ]:
!docker run -d -p 8000:8000 -e RETRAIN_API_KEY="aks"  -e GH_PAT="add own personal access token here" --name iris_api_service iris-classifier:latest 

edc78de7ec69808744c48df323fb938e659beecefb770470eb5f74de793f0039


## Step 5: Test the Running API

Our API should now be live and accessible at `http://localhost:8000`. We can send a `POST` request with sample data using `curl` to test the `/predict` endpoint.

You should receive a JSON response with the predicted Iris species.

In [6]:
import requests

url = "http://localhost:8000/predict"
data = {
    "sepal_length": 5.1,
    "sepal_width": 3.5,
    "petal_length": 1.4,
    "petal_width": 0.2
}

response = requests.post(url, json=data)
print(response.json())



{'predicted_species': 'setosa', 'predicted_label': 0, 'prediction_confidence': 0.9797402531111016}


## Step 6: Stop and Remove the Container (Cleanup)

Once you are finished testing, you can stop and remove the running container to free up resources.

In [ ]:
!docker stop iris_api_service
!docker rm iris_api_service